In [2]:
!pip install groq gradio

In [ ]:
import os
import json
import gradio as gr
from groq import Groq
from datetime import datetime

class CurrentAffairsQA:
    def __init__(self):
        self.client = None
        self.api_key_verified = False

    def verify_api_key(self, api_key):
        """Verify if the Groq API key is valid"""
        if not api_key or api_key.strip() == "":
            return False, "❌ Please enter an API key"

        try:
            # Create a test client
            test_client = Groq(api_key=api_key.strip())

            # Make a simple test request
            response = test_client.chat.completions.create(
                messages=[{"role": "user", "content": "Hi"}],
                model="llama-3.3-70b-versatile",
                max_tokens=10
            )

            # If we get here, the API key is valid
            self.client = test_client
            self.api_key_verified = True
            return True, "✅ API Key verified successfully!"

        except Exception as e:
            self.api_key_verified = False
            return False, f"❌ API Key verification failed: {str(e)}"

    def answer_question(self, question, api_key):
        """Answer current affairs questions using Groq API"""

        # Verify API key first if not already verified
        if not self.api_key_verified or self.client is None:
            verified, message = self.verify_api_key(api_key)
            if not verified:
                return message

        if not question or question.strip() == "":
            return "❌ Please enter a question"

        try:
            # System prompt to ensure only current affairs questions are answered
            system_prompt = f"""You are a Current Affairs Expert Assistant. Today's date is {datetime.now().strftime('%B %d, %Y')}.

IMPORTANT RULES:
1. ONLY answer questions related to current affairs, news, recent events, politics, economics, sports, technology news, and world events.
2. If a question is NOT about current affairs (e.g., general knowledge, math problems, coding, personal advice, entertainment trivia not related to recent events), respond EXACTLY with: "I can only answer questions related to current affairs. Please ask about recent news, events, or developments."
3. For current affairs questions, provide accurate, concise, and informative answers.
4. Focus on events from the past few years, with emphasis on recent developments.
5. If you're unsure whether something is current affairs, err on the side of caution and only answer if it relates to recent news or events.

Examples of VALID questions:
- What are the latest developments in AI regulation?
- Who won the recent elections in [country]?
- What's happening with the economy?
- Recent sports championship results
- Latest climate change policies

Examples of INVALID questions (respond with the rejection message):
- What is 2+2?
- How do I code in Python?
- Write me a story
- What's the meaning of life?
- How to cook pasta?"""

            # Make API call to Groq
            response = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": question}
                ],
                model="llama-3.3-70b-versatile",
                temperature=0.7,
                max_tokens=1024
            )

            answer = response.choices[0].message.content
            return answer

        except Exception as e:
            return f"❌ Error processing question: {str(e)}"

# Initialize the QA system
qa_system = CurrentAffairsQA()

def verify_key_interface(api_key):
    """Interface function for API key verification"""
    verified, message = qa_system.verify_api_key(api_key)
    return message

def ask_question_interface(question, api_key):
    """Interface function for asking questions"""
    return qa_system.answer_question(question, api_key)

# Create Gradio Interface
with gr.Blocks(theme=gr.themes.Soft(), title="Current Affairs Q&A") as demo:
    gr.Markdown(
        """
        # 📰 Current Affairs Q&A Assistant
        ### Powered by Groq API

        Ask questions about recent news, events, politics, economics, sports, and world affairs.
        **Note:** This assistant only answers current affairs questions.
        """
    )

    with gr.Row():
        with gr.Column(scale=2):
            api_key_input = gr.Textbox(
                label="🔑 Groq API Key",
                placeholder="Enter your Groq API key here...",
                type="password",
                lines=1
            )
        with gr.Column(scale=1):
            verify_btn = gr.Button("✓ Verify API Key", variant="primary", size="lg")

    verification_status = gr.Textbox(
        label="Verification Status",
        interactive=False,
        lines=1
    )

    gr.Markdown("---")

    with gr.Row():
        question_input = gr.Textbox(
            label="❓ Your Question",
            placeholder="e.g., What are the latest developments in renewable energy?",
            lines=3
        )

    with gr.Row():
        submit_btn = gr.Button("🚀 Submit Question", variant="primary", size="lg")
        clear_btn = gr.Button("🗑️ Clear", variant="secondary")

    answer_output = gr.Textbox(
        label="📝 Answer",
        lines=10,
        interactive=False
    )

    gr.Markdown(
        """
        ---
        ### 💡 Tips:
        - Get your Groq API key from: [console.groq.com](https://console.groq.com)
        - Verify your API key before asking questions
        - Ask about recent news, events, and developments
        - Questions unrelated to current affairs will not be answered
        """
    )

    # Event handlers
    verify_btn.click(
        fn=verify_key_interface,
        inputs=[api_key_input],
        outputs=[verification_status]
    )

    submit_btn.click(
        fn=ask_question_interface,
        inputs=[question_input, api_key_input],
        outputs=[answer_output]
    )

    clear_btn.click(
        fn=lambda: ("", ""),
        inputs=[],
        outputs=[question_input, answer_output]
    )

    # Example questions
    gr.Examples(
        examples=[
            ["What are the latest developments in artificial intelligence?"],
            ["Who won the recent FIFA World Cup?"],
            ["What's happening with global climate policies?"],
            ["Latest updates on space exploration missions"],
            ["Recent economic trends in major economies"]
        ],
        inputs=[question_input]
    )

# Launch the app
if __name__ == "__main__":
    # For Google Colab, use share=True to get a public URL
    demo.launch(
        share=True,  # Creates a public link for Colab
        debug=True,
        server_name="0.0.0.0",  # Allows external connections
        server_port=7860
    )

/tmp/ipython-input-2146638937.py:104: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), title="Current Affairs Q&A") as demo:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1bca5dbc23092a08ae.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
